In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def seq_model(params):
    
    inp_dim = params['inp_dim']
    out_dim = params['out_dim']
    hid_dim = params['hid_dim']
    n_hidden = len(hid_dim)+1
    
    W,B = [],[]

    with tf.variable_scope("weights"):
        for i in range(n_hidden):
            if(i==0):
                W.append(tf.get_variable(name="w"+str(i), 
                                         initializer=tf.glorot_normal_initializer(dtype=tf.float64), 
                                         shape=(inp_dim,hid_dim[i]), 
                                         dtype=tf.float64))
                B.append(tf.get_variable(name="b"+str(i), 
                                         initializer=tf.glorot_normal_initializer(dtype=tf.float64), 
                                         shape=(1,hid_dim[i]), 
                                         dtype=tf.float64))
            elif(i<n_hidden-1):
                W.append(tf.get_variable(name="w"+str(i), 
                                         initializer=tf.glorot_normal_initializer(dtype=tf.float64), 
                                         shape=(hid_dim[i-1],hid_dim[i]), 
                                         dtype=tf.float64))
                B.append(tf.get_variable(name="b"+str(i), 
                                         initializer=tf.glorot_normal_initializer(dtype=tf.float64), 
                                         shape=(1,hid_dim[i]), 
                                         dtype=tf.float64))
            else:
                W.append(tf.get_variable(name="w"+str(i), 
                                         initializer=tf.glorot_normal_initializer(dtype=tf.float64), 
                                         shape=(hid_dim[i-1],out_dim), 
                                         dtype=tf.float64))
                B.append(tf.get_variable(name="b"+str(i), 
                                         initializer=tf.glorot_normal_initializer(dtype=tf.float64), 
                                         shape=(1,out_dim), 
                                         dtype=tf.float64))
        return W,B
    
def graph_gen(x,y):
    
    params = {'inp_dim':4, 'out_dim':3, 'hid_dim':[8,8,8,8]}
    W,B = seq_model(params)
    n_hidden = len(params['hid_dim'])+1
    
    with tf.variable_scope("weights", reuse=tf.AUTO_REUSE):
        
        for i in range(n_hidden):
            if(i==0):
                l = tf.nn.relu(tf.add(tf.matmul(x,W[i]),B[i]))
            elif(i<n_hidden-1):
                l = tf.nn.relu(tf.add(tf.matmul(l,W[i]),B[i]))
            else:
                out = tf.nn.relu(tf.add(tf.matmul(l,W[i]),B[i]))
        
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=out, labels=y)
        optim = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
        acc = tf.reduce_mean(tf.cast((tf.equal(tf.argmax(y,1), tf.argmax(out,1))), dtype=tf.float64))
        
        for i in range(len(W)):
            tf.summary.histogram("w"+str(i), W[i])
        tf.summary.scalar("loss", tf.reduce_mean(loss))
        tf.summary.scalar("acc", acc)
        
        return loss, optim, acc
    
def model_run(input_train, output_train, input_test, output_test, epochs=None):

    with tf.Session() as sess:

        with tf.variable_scope("inputs"):
            x = tf.placeholder(dtype=tf.float64, shape=(None,4))
            y = tf.placeholder(dtype=tf.float64, shape=(None,3))
    
        loss, optim, acc = graph_gen(x,y)

        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter('./logs2/', sess.graph)

        sess.run(tf.global_variables_initializer())

        for ep in range(epochs):
            a, s, l, _ = sess.run([acc, merged, loss, optim], feed_dict={x:input_train, y:output_train})
            print("Epoch : ", ep, "Loss : ", np.mean(l), "Accuracy : ", a)
            writer.add_summary(s, ep)
        
        a, l = sess.run([acc, loss], feed_dict={x:input_test, y:output_test})
        print("\n\nTesting Loss : ", np.mean(l), "Testing Accuracy : ", a)

In [3]:
data = pd.read_csv("../Python/Datasets/Iris.csv", index_col='Id').sample(frac=1).reset_index(drop=True)
data = pd.concat([data, pd.get_dummies(data['Species'])], axis=1).drop(columns=['Species'])
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,6.9,3.2,5.7,2.3,0,0,1
1,5.8,2.7,4.1,1.0,0,1,0
2,5.2,3.5,1.5,0.2,1,0,0
3,5.8,2.7,5.1,1.9,0,0,1
4,6.7,2.5,5.8,1.8,0,0,1


In [4]:
split = int(0.7*data.shape[0])
# inputs = np.random.random(size=(10,4))
# outputs = np.eye(3)[np.random.choice(3, 10)]

input_train, output_train, input_test, output_test = data.values[:split,:-3], data.values[:split,-3:], data.values[split:,:-3], data.values[split:,-3:]

In [5]:
model_run(input_train, output_train, input_test, output_test, epochs=1000)

Epoch :  0 Loss :  1.3927426864651833 Accuracy :  0.29523809523809524
Epoch :  1 Loss :  1.3870288164837392 Accuracy :  0.29523809523809524
Epoch :  2 Loss :  1.3814175531860138 Accuracy :  0.29523809523809524
Epoch :  3 Loss :  1.3758966117341243 Accuracy :  0.29523809523809524
Epoch :  4 Loss :  1.370456636531413 Accuracy :  0.29523809523809524
Epoch :  5 Loss :  1.3650964093205062 Accuracy :  0.29523809523809524
Epoch :  6 Loss :  1.3598132480984477 Accuracy :  0.29523809523809524
Epoch :  7 Loss :  1.3545993428342211 Accuracy :  0.29523809523809524
Epoch :  8 Loss :  1.3494851065339444 Accuracy :  0.29523809523809524
Epoch :  9 Loss :  1.34444761513102 Accuracy :  0.29523809523809524
Epoch :  10 Loss :  1.3394711700695177 Accuracy :  0.29523809523809524
Epoch :  11 Loss :  1.334564763884886 Accuracy :  0.29523809523809524
Epoch :  12 Loss :  1.3297957663691802 Accuracy :  0.29523809523809524
Epoch :  13 Loss :  1.3250960718488924 Accuracy :  0.29523809523809524
Epoch :  14 Loss :  

Epoch :  139 Loss :  0.8113736275023955 Accuracy :  0.38095238095238093
Epoch :  140 Loss :  0.8083366227425726 Accuracy :  0.38095238095238093
Epoch :  141 Loss :  0.805338130051814 Accuracy :  0.38095238095238093
Epoch :  142 Loss :  0.8023072362589064 Accuracy :  0.38095238095238093
Epoch :  143 Loss :  0.7993266099882289 Accuracy :  0.38095238095238093
Epoch :  144 Loss :  0.7964251821819663 Accuracy :  0.4
Epoch :  145 Loss :  0.793562093432375 Accuracy :  0.41904761904761906
Epoch :  146 Loss :  0.7907367998933251 Accuracy :  0.4666666666666667
Epoch :  147 Loss :  0.7879527911532486 Accuracy :  0.5428571428571428
Epoch :  148 Loss :  0.7852069420158814 Accuracy :  0.6
Epoch :  149 Loss :  0.782475851525692 Accuracy :  0.6190476190476191
Epoch :  150 Loss :  0.7797513934115473 Accuracy :  0.6571428571428571
Epoch :  151 Loss :  0.7770394204005772 Accuracy :  0.6666666666666666
Epoch :  152 Loss :  0.7743386509194056 Accuracy :  0.6666666666666666
Epoch :  153 Loss :  0.7716539428

Epoch :  270 Loss :  0.4904757527119647 Accuracy :  0.8857142857142857
Epoch :  271 Loss :  0.4883798953213337 Accuracy :  0.8857142857142857
Epoch :  272 Loss :  0.4862710536596933 Accuracy :  0.8857142857142857
Epoch :  273 Loss :  0.48414956627871775 Accuracy :  0.8857142857142857
Epoch :  274 Loss :  0.48201018955128533 Accuracy :  0.8857142857142857
Epoch :  275 Loss :  0.47985336135782825 Accuracy :  0.8952380952380953
Epoch :  276 Loss :  0.4776608096963302 Accuracy :  0.8952380952380953
Epoch :  277 Loss :  0.4754469996643 Accuracy :  0.8952380952380953
Epoch :  278 Loss :  0.47321305832508365 Accuracy :  0.9047619047619048
Epoch :  279 Loss :  0.4709574325018045 Accuracy :  0.9047619047619048
Epoch :  280 Loss :  0.46867973355838766 Accuracy :  0.9047619047619048
Epoch :  281 Loss :  0.46637971936868255 Accuracy :  0.9047619047619048
Epoch :  282 Loss :  0.4640573399334668 Accuracy :  0.9047619047619048
Epoch :  283 Loss :  0.4617129859630466 Accuracy :  0.9047619047619048
Epo

Epoch :  400 Loss :  0.1343578404079932 Accuracy :  0.9714285714285714
Epoch :  401 Loss :  0.13292066717332632 Accuracy :  0.9714285714285714
Epoch :  402 Loss :  0.13151263786560582 Accuracy :  0.9714285714285714
Epoch :  403 Loss :  0.13013303496938916 Accuracy :  0.9714285714285714
Epoch :  404 Loss :  0.12878120013832797 Accuracy :  0.9714285714285714
Epoch :  405 Loss :  0.12745698621114632 Accuracy :  0.9714285714285714
Epoch :  406 Loss :  0.12615998219294405 Accuracy :  0.9714285714285714
Epoch :  407 Loss :  0.12488896509543351 Accuracy :  0.9714285714285714
Epoch :  408 Loss :  0.12364325638514002 Accuracy :  0.9714285714285714
Epoch :  409 Loss :  0.12242227683063023 Accuracy :  0.9714285714285714
Epoch :  410 Loss :  0.121226174192387 Accuracy :  0.9714285714285714
Epoch :  411 Loss :  0.12005369023804033 Accuracy :  0.9714285714285714
Epoch :  412 Loss :  0.11890416677637908 Accuracy :  0.9714285714285714
Epoch :  413 Loss :  0.1177769891920032 Accuracy :  0.9714285714285

Epoch :  539 Loss :  0.054751138726426216 Accuracy :  0.9904761904761905
Epoch :  540 Loss :  0.054521313940788886 Accuracy :  0.9904761904761905
Epoch :  541 Loss :  0.054293471033729224 Accuracy :  0.9904761904761905
Epoch :  542 Loss :  0.054067614240951065 Accuracy :  0.9904761904761905
Epoch :  543 Loss :  0.05384368843261063 Accuracy :  0.9904761904761905
Epoch :  544 Loss :  0.05362171755268964 Accuracy :  0.9904761904761905
Epoch :  545 Loss :  0.05340175614219269 Accuracy :  0.9904761904761905
Epoch :  546 Loss :  0.05318363860064891 Accuracy :  0.9904761904761905
Epoch :  547 Loss :  0.0529673468946977 Accuracy :  0.9904761904761905
Epoch :  548 Loss :  0.05275292365134406 Accuracy :  0.9904761904761905
Epoch :  549 Loss :  0.052540272194806815 Accuracy :  0.9904761904761905
Epoch :  550 Loss :  0.05232941565632929 Accuracy :  0.9904761904761905
Epoch :  551 Loss :  0.05212036011408259 Accuracy :  0.9904761904761905
Epoch :  552 Loss :  0.05191312210865605 Accuracy :  0.99047

Epoch :  683 Loss :  0.03594176266530849 Accuracy :  0.9904761904761905
Epoch :  684 Loss :  0.03587304753180711 Accuracy :  0.9904761904761905
Epoch :  685 Loss :  0.03580478754248659 Accuracy :  0.9904761904761905
Epoch :  686 Loss :  0.03573697596165737 Accuracy :  0.9904761904761905
Epoch :  687 Loss :  0.03566960975624858 Accuracy :  0.9904761904761905
Epoch :  688 Loss :  0.03560268384991621 Accuracy :  0.9904761904761905
Epoch :  689 Loss :  0.03553621026757191 Accuracy :  0.9809523809523809
Epoch :  690 Loss :  0.03547016750736701 Accuracy :  0.9809523809523809
Epoch :  691 Loss :  0.03540455274651327 Accuracy :  0.9809523809523809
Epoch :  692 Loss :  0.03533936047904413 Accuracy :  0.9809523809523809
Epoch :  693 Loss :  0.03527458638959718 Accuracy :  0.9809523809523809
Epoch :  694 Loss :  0.03521022563690304 Accuracy :  0.9809523809523809
Epoch :  695 Loss :  0.03514627400266735 Accuracy :  0.9809523809523809
Epoch :  696 Loss :  0.03508272726420498 Accuracy :  0.980952380

Epoch :  830 Loss :  0.029103764839786654 Accuracy :  0.9809523809523809
Epoch :  831 Loss :  0.0290714425723122 Accuracy :  0.9809523809523809
Epoch :  832 Loss :  0.029039231009637748 Accuracy :  0.9809523809523809
Epoch :  833 Loss :  0.029007128686158432 Accuracy :  0.9809523809523809
Epoch :  834 Loss :  0.02897518738106715 Accuracy :  0.9809523809523809
Epoch :  835 Loss :  0.02894336045430026 Accuracy :  0.9809523809523809
Epoch :  836 Loss :  0.028911644241985734 Accuracy :  0.9809523809523809
Epoch :  837 Loss :  0.02888003737634121 Accuracy :  0.9809523809523809
Epoch :  838 Loss :  0.028848537326751057 Accuracy :  0.9809523809523809
Epoch :  839 Loss :  0.028817143077060858 Accuracy :  0.9809523809523809
Epoch :  840 Loss :  0.028785854268661856 Accuracy :  0.9809523809523809
Epoch :  841 Loss :  0.028754668451396105 Accuracy :  0.9809523809523809
Epoch :  842 Loss :  0.028723585747697584 Accuracy :  0.9809523809523809
Epoch :  843 Loss :  0.02869260455469518 Accuracy :  0.9

Epoch :  983 Loss :  0.025152934180334145 Accuracy :  1.0
Epoch :  984 Loss :  0.02513184324007644 Accuracy :  1.0
Epoch :  985 Loss :  0.0251107896584641 Accuracy :  1.0
Epoch :  986 Loss :  0.02508978983120326 Accuracy :  1.0
Epoch :  987 Loss :  0.025068828386471648 Accuracy :  1.0
Epoch :  988 Loss :  0.02504790235706166 Accuracy :  1.0
Epoch :  989 Loss :  0.025027012196191536 Accuracy :  1.0
Epoch :  990 Loss :  0.025006158716562737 Accuracy :  1.0
Epoch :  991 Loss :  0.024985353370132263 Accuracy :  1.0
Epoch :  992 Loss :  0.02496458565843064 Accuracy :  1.0
Epoch :  993 Loss :  0.024943851506349028 Accuracy :  1.0
Epoch :  994 Loss :  0.024923150765580437 Accuracy :  1.0
Epoch :  995 Loss :  0.02490248947254152 Accuracy :  1.0
Epoch :  996 Loss :  0.024881870612288692 Accuracy :  1.0
Epoch :  997 Loss :  0.024861285953351953 Accuracy :  1.0
Epoch :  998 Loss :  0.024840735654890492 Accuracy :  1.0
Epoch :  999 Loss :  0.024820220219236144 Accuracy :  1.0


Testing Loss :  0.1